In [1]:
#pip install pymorphy2

In [2]:
import requests 

In [3]:
import docx
import spacy
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

# Load spaCy Russian tokenizer
nlp = spacy.blank("ru")

# Load the Ruberta model and tokenizer from Hugging Face transformers
model_name = "cointegrated/rubert-tiny2"  # Pre-trained Ruberta model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def read_docx(file_path):
    # Open the Word document
    doc = docx.Document(file_path)
    
    # Extract text content from the document
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    
    # Join all paragraphs into a single string
    document_text = "\n".join(full_text)
    return document_text

def chunking(document_text, max_chunk_len=600):
    # Split the document into chunks
    doc = nlp(document_text)
    chunks = []
    current_chunk = []

    for token in doc:
        if len(current_chunk) + len(token.text) > max_chunk_len:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

        current_chunk.append(token.text)

    if current_chunk:
        chunks.append(" ".join(current_chunk))



    return chunks


file_path = "main_doc.docx"
document_text = read_docx(file_path)
chunks = chunking(document_text)

In [4]:
import numpy as np
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemma(text):
    lemmatized_words = []
    for word in text:
        parsed_word = morph.parse(word)[0]
        lemmatized_words.append(parsed_word.normal_form)
    return lemmatized_words


def jaccard_similarity(s1, s2):
    morph = pymorphy2.MorphAnalyzer()
    s1 = s1.lower().split(" ")
    s2 = s2.lower().split(" ")
    s1 = lemma(s1)
    s2= lemma(s2)
    set1, set2 = set(s1), set(s2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union




def jacc_search(query, chunks):
    distances = []

    for chunk in chunks:
        dist = jaccard_similarity(query, chunk)
        distances.append(dist)

    # Find indices of the top 3 closest chunks (smallest distances)
    closest_indices = np.argsort(distances)[::-1][:3]
    closest_chunks = [chunks[idx] for idx in closest_indices]

    return closest_chunks, distances,closest_indices



query = 'За что отвечает академический руководитель?'
prompt = {
    "modelUri": "gpt://folder-id/yandexgpt-lite",
    "completionOptions": {
        "stream": False,
        "temperature": 0.6,
        "maxTokens": "2000"
    },
    "messages": [
        {
            "role": "system",
            "text": "ты умный помошник студента, постарайся ответить на поставленный вопрос"
        },
        {
            "role": "user",
            "text": query
        }
    ]
}


url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Api-Key your key"
}

response = requests.post(url, headers=headers, json=prompt)
result = response.text
response_data = response.json()
assistant_text = response_data['result']['alternatives'][0]['message']['text']
print(assistant_text)

# Yandex GPT answer to use for query expansion. 

Академический руководитель — это наставник, который помогает студенту в процессе обучения в вузе. Он может выполнять следующие функции:

1. Консультирование студента по вопросам выбора образовательной траектории и составления индивидуального плана обучения.

2. Помощь в определении и корректировке направлений учебной и научной деятельности студента.

3. Сопровождение студента в процессе решения административных вопросов, связанных с обучением.

4. Поддержка в вопросах планирования и организации учебного процесса, а также в решении любых возникающих проблем.

5. Мотивация студента к активному участию в учебной и внеучебной жизни вуза.

6. Помощь в профессиональном развитии и карьерном росте студента.


In [5]:
closest_chunks,dist,cl = jacc_search(query+assistant_text,chunks)
i=1
print(dist)
print()
print(cl)
print("Closest chunks to the query:")
for idx, chunk in enumerate(closest_chunks, start=1):
    print('chunk ',i)
    print()
    i+=1
    print(f"{idx}. {chunk}")
    print()

[0.09486166007905138, 0.07169811320754717, 0.06909090909090909, 0.08178438661710037, 0.06785714285714285, 0.07142857142857142, 0.08870967741935484, 0.06589147286821706, 0.08088235294117647, 0.06909090909090909, 0.09090909090909091, 0.06227106227106227, 0.0975609756097561, 0.09829059829059829, 0.07142857142857142, 0.07380073800738007, 0.07630522088353414, 0.07894736842105263, 0.06181818181818182, 0.07692307692307693, 0.07194244604316546, 0.07636363636363637, 0.072992700729927, 0.05555555555555555, 0.06273062730627306, 0.0695970695970696, 0.08148148148148149, 0.0684931506849315, 0.06015037593984962, 0.05925925925925926, 0.08778625954198473, 0.08461538461538462, 0.08015267175572519, 0.06909090909090909, 0.07608695652173914, 0.05555555555555555, 0.08833922261484099, 0.07633587786259542, 0.0683453237410072, 0.0743801652892562, 0.07335907335907337, 0.09913793103448276, 0.05982905982905983]

[41 13 12]
Closest chunks to the query:
chunk  1

1. выставлении промежуточных/ окончательных оценок с

In [6]:
context = str(closest_chunks)

prompt = {
    "modelUri": "gpt://folder-id/yandexgpt-lite",
    "completionOptions": {
        "stream": False,
        "temperature": 0.6,
        "maxTokens": "2000"
    },
    "messages": [
        {
            "role": "system",
            "text": "ты умный помошник студента, используй документы ниже, чтобы ответить на поставленный вопрос, если в документах этой информации нет, то так и скажи \
             вот ДОКУМЕНТЫ: " + context
        },
        {
            "role": "user",
            "text": query
        }
    ]
}


url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Api-Key your key"
}

response = requests.post(url, headers=headers, json=prompt)
result = response.text
response_data = response.json()
assistant_text = response_data['result']['alternatives'][0]['message']['text']
print(assistant_text)

Согласно приведённым фрагментам документов, академический руководитель отвечает за:

* подготовку предложений по переводу студентов образовательной программы на следующий курс;

* организацию работы академических консультантов и взаимодействие с преподавателями;

* взаимодействие с другими образовательными организациями и подразделениями НИУ ВШЭ;

* анализ качества подготовки студентов и внесение предложений по улучшению образовательного процесса.

Также академический руководитель может выполнять другие обязанности, которые будут установлены локальными нормативными актами НИУ ВШЭ.
